<font size="8"> **Accessing sea ice concentration data from NASA Goddard** </font>  
This notebook access the [NASA Goddard-merged Near Real Time NOAA/NSIDC Climate Data Record of Passive Microwave Sea Ice Concentration](https://climatedataguide.ucar.edu/climate-data/sea-ice-concentration-data-nasa-goddard-and-nsidc-based-nasa-team-algorithm) (version 3) dataset available in Gadi.  
  
The notebook also applies some corrections to the observational data to cover missing data for some dates between 1979 and 1988.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [4]:
import os
#Ensure you update this filepath if you have saved these notebooks elsewhere in your machine
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading relevant libraries

In [5]:
#Loading and manipulating data
import cosima_cookbook as cc
import xarray as xr
import numpy as np
import pandas as pd
import re
# import datetime as dt
from glob import glob
from dask.distributed import Client
# import geopandas as gp
# import string
# from itertools import cycle
# import xesmf as xe
# import scipy.stats as ss

#Analysis module
import UsefulFunctions as uf

#Packages for plotting
import matplotlib.pyplot as plt
# import matplotlib.ticker as mticker
# import matplotlib.colors as mcolors
# from matplotlib.ticker import AutoMinorLocator, MultipleLocator
# from mpl_toolkits.axes_grid1.inset_locator import inset_axes
# import cmocean as cm
# import cartopy.crs as ccrs
# import cartopy.feature as cft
# import matplotlib.gridspec as gridspec
#Show plots produced by matplotlib inside this Jupyter notebook
%matplotlib inline

## Parallelising work

In [6]:
client = Client()

# Defining dictionary of useful variables
In this dictionary we will define a variables that will be used multiple times throughout this notebook to avoid repetition. It will mostly contain paths to folders where intermediate or final outputs will be stored.

In [7]:
varDict =  {#'var_id': 'aice',
            # 'model': 'ACCESS-OM2-01',
            # 'expt': '01deg_jra55v140_iaf_cycle4', # ACCESS-OM2-01 cycle 2
            # 'freq': '1 daily',
            # 'start_date': range(1981, 2014, 1),
            # 'offset': np.nan,
            # 'long_name': 'sea ice seasonality',
            # 'short_name': 'SIS',
            # 'unit': 'days',
            # 'CICE_data': True,
            'Observations': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs'}

# Accessing sea ice concentration data from observations
Data for the ACCESS-OM2-01 model and satellite observations are available in GADI. The observational data has some missing data between 1979 and 1988. We will interpolate data for the missing periods as described in the next section.

In [46]:
#Observational data file path to main folder location
ObsDir = '/g/data3/hh5/tmp/cosima/observations/NOAA/G02202_V3'

#Get a list of all files inside the year folders
obsSHFileList = sorted(glob(os.path.join(ObsDir, 'south/daily/*/*'), recursive = True))

# Load all files that are within the years of interest
for year in np.arange(1978, 2020):
    #Create an empty list to hold all filepaths
    fileList = []
    for file in obsSHFileList:
        #Extracting full date (8 digits long) and then comparing with year (first 4 digits)
        if str(year) in re.search("[0-9]{8}", file).group(0)[:4]:
            #Only one variable 'goddard_merged_seaice_conc' is loaded as it is the only one of interest
            dataarray = xr.open_dataset(file, decode_times = False, autoclose = True)['goddard_merged_seaice_conc']
            fileList.append(dataarray)
    #Create a new data array with all time steps
    fileList = xr.concat(fileList, dim = 'time', coords = 'minimal')    
    if 'time' in fileList.coords:
        #Apply the same time units given in time dimension to time coordinates
        time_units = fileList.time.units
        #Decode date time data to CF conventions
        decoded_time = xr.conventions.times.decode_cf_datetime(fileList.time, time_units)
        fileList.coords['time'] = ('time', decoded_time,
                                   {'long_name': 'time', 'decoded_using': time_units})
    # replace values outside valid range (0-1) with nan
    fileList = fileList.where((fileList >= 0) & (fileList <= 1), np.nan)
    
    #Save data array in the observations folder
    os.makedirs(varDict['Observations'], exist_ok = True)
    fileList.to_netcdf(os.path.join(varDict['Observations'], f'sea_ice_conc_obs_{str(year)}.nc'))

# Dealing with missing dates in sea ice concentration from observations
Data is not available for some periods in the observational data. We applied two different corrections as described in [Masson et al 2013](https://doi.org/10.1371/journal.pone.0064756):  
1. Between January 1979 and November 1987, days of missing data were interpolated from adjoining days  
2. Between December 1987 and January 1988, missing days were replaced with daily climatology (1979-2009)  

## Correcting data between Jan 1979 and Nov 1987
First, we load the data for this period as a data array.

In [9]:
#Getting a list of all SIC observations saved in previous step
obs_list = sorted(glob(os.path.join(varDict['Observations'], '*.nc')))

#Loading data for the period between 1979 and 1987 - First correction
obs_da = []
#Looking for files between period of interest and loading as data array
for yr in np.arange(1978, 1988):
    [obs_da.append(xr.open_dataarray(f)) for f in obs_list if str(yr) in f]

#Concatening all files together
obs_da = xr.concat(obs_da, dim = 'time').sortby('time')

Now we will identify the missing dates, interpolate data from adjoining days and include these in a corrected dataset.

In [10]:
#Get list of dates not included in observational data
obs_time = obs_da.time.values
complete_time = pd.date_range(obs_da.time.values.min(), obs_da.time.values.max()).values

#Find missing days
miss_days = list(set(obs_time) ^ set(complete_time))

#Create an empty list to hold interpolated days
int_data = []

#Interpolate only days that are missing
for md in miss_days:
    #Select the day prior and the day after missing date
    int_range = obs_da.sel(time = slice(md-np.timedelta64(1, 'D'), md+np.timedelta64(1, 'D')))
    #Perform interpolation
    int_range = int_range.interp(time = pd.date_range(md - np.timedelta64(1, 'D'), md + np.timedelta64(1, 'D')))
    #Save interpolated missing date in a list
    int_data.append(int_range.sel(time = md))

#Create a new array with missing dates
obs_corr = xr.concat([obs_da, xr.concat(int_data, dim = 'time')], dim = 'time').sortby('time')

#Remove variables no longer in use
del obs_da, obs_time, complete_time, miss_days

We will merge the interpolated data with the original observational data from 1988 to 2009.

In [11]:
#Create empty list to hold file list
obs_da = []

#Load data arrays
for yr in np.arange(1988, 2010):
    [obs_da.append(xr.open_dataarray(f)) for f in obs_list if str(yr) in f]

#Create a new data array to calculate climatology. Joining original files just loaded and the interpolated data
obs_clim = xr.concat([obs_corr.where(obs_corr.time.dt.year >= 1979, drop = True),
                      xr.concat(obs_da, dim = 'time')], dim = 'time')

### Saving corrected data between Jan 1979 and Nov 1987

In [14]:
for yr in np.arange(1979, 1988):
    obs_clim.sel(time = str(yr)).to_netcdf(os.path.join(varDict['Observations'], f'sea_ice_conc_obs_{str(yr)}.nc'))

Finally, we will check that there are no other missing days outside the period that we corrected in the previous steps. If there are, we will interpolate from adjoining dates.

In [15]:
#Get list of dates not included in observational data
obs_time = obs_clim.time.values
complete_time = pd.date_range(obs_clim.time.values.min(), obs_clim.time.values.max()).values

#Find missing days
miss_days = list(set(obs_time) ^ set(complete_time))

#Checking that there are no missing dates outside 1987-1988
int_dates = []
[int_dates.append(i) for i in miss_days if str(pd.Timestamp(i).year) not in ['1987', '1988']]

#Creating missing list to store data
int_da = []
#Looping through missing dates
for d in int_dates:
    int_range = obs_clim.sel(time = slice(d-np.timedelta64(1, 'D'), d+np.timedelta64(1, 'D')))
    #Perform interpolation
    int_range = int_range.interp(time = pd.date_range(d-np.timedelta64(1, 'D'), d+np.timedelta64(1, 'D')))
    #Save interpolated missing date in a list
    int_da.append(int_range.sel(time = d))
    #Save data for years where data has been interpolated
    yr = str(pd.Timestamp(i).year)
    obs_clim.sel(time = yr).to_netcdf(os.path.join(varDict['Observations'], f'sea_ice_conc_obs_{str(yr)}.nc'))

#Add interpolated data
obs_clim = xr.concat([obs_clim, xr.concat(int_data, dim = 'time')], dim = 'time').sortby('time')

## Correcting data between Dec 1987 and Jan 1988
Any missing dates between this period will be replaced by the daily climatology calculated from the dataset corrected in the previous step.  
  
First, we calculate the daily climatological mean.

In [25]:
#Calculate climatology
daily_clim = obs_clim.groupby('time.dayofyear').mean('time')

Replacing missing days with daily climatology (1979-2009).

In [26]:
#Get list of dates not included in observational data
obs_time = obs_clim.time.values
complete_time = pd.date_range(obs_clim.time.values.min(), obs_clim.time.values.max()).values

#Find missing days
miss_days = list(set(obs_time) ^ set(complete_time))

#Replace missing dates with climatology
corr_dates = []
for md in miss_days:
    #Add a day to dates in 1987 because daily climatology includes leap day
    if '1987' in str(pd.to_datetime(md)):
        ind = pd.Timestamp(md).timetuple().tm_yday+1
    else:
        ind = pd.Timestamp(md).timetuple().tm_yday
    corr_dates.append(daily_clim.sel(dayofyear = ind).drop('dayofyear').expand_dims({'time': [md]}))

#Save everything in corrected data array
obs_clim = xr.concat([obs_clim, xr.concat(corr_dates, dim = 'time')], dim = 'time').sortby('time')

### Saving corrected data between 1987 and 1988

In [28]:
for yr in np.arange(1987, 1989):
    obs_clim.sel(time = str(yr)).to_netcdf(os.path.join(varDict['Observations'], f'sea_ice_conc_obs_{str(yr)}.nc'))

## Calculating monthly means from corrected observational dataset
We will use mean monthly data to model the distribution of crabeater seals. Thus, we will calculate monthly means before storing the data to disk.

In [163]:
#Defining folder where monthly means will be saved
folder_out = os.path.join(varDict['Observations'], 'monthly')
#Ensure folder exists
os.makedirs(folder_out, exist_ok = True)

#Loading each file - Calculate monthly means and save outputs
for f in obs_list:
    #Open yearly SIC file
    da = xr.open_dataarray(f)
    #Getting year in file
    yr = da.time.dt.year
    #Ensuring there is data for only one year in the file
    if len(pd.unique(yr)) == 1:
        yr = pd.unique(yr)[0]
    else:
        print('More than one year included in data: ', pd.unique(yr))
    #Calculate monthly mean
    da = da.groupby('time.month').mean()
    #Creating monthly date to be included in data array (mid-month)
    dates = [pd.to_datetime(f'{yr}-{m}-15') for m in da.month.values]
    #Renaming month dimension as time
    da = da.rename({'month': 'time'})
    #Using corrected dates
    da['time'] = dates
    #Saving outputs
    da.to_netcdf(os.path.join(folder_out, f'monthly_mean_sic_obs_{str(yr)}.nc'))

In [164]:
sic_monthly_obs = xr.open_mfdataset(sorted(glob(os.path.join(varDict['Observations'], 'monthly/*.nc'))))
sic_monthly_obs

<xarray.Dataset>
Dimensions:                     (ygrid: 332, xgrid: 316, time: 494)
Coordinates:
  * ygrid                       (ygrid) float32 4.338e+06 ... -3.938e+06
  * xgrid                       (xgrid) float32 -3.938e+06 ... 3.938e+06
    longitude                   (ygrid, xgrid) float64 dask.array<chunksize=(332, 316), meta=np.ndarray>
    latitude                    (ygrid, xgrid) float64 dask.array<chunksize=(332, 316), meta=np.ndarray>
  * time                        (time) datetime64[ns] 1978-11-15 ... 2019-12-15
Data variables:
    goddard_merged_seaice_conc  (time, ygrid, xgrid) float64 dask.array<chunksize=(2, 332, 316), meta=np.ndarray>